In [2]:
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
from elsapy.utils import recast_df
import pandas as pd
import json

## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

## Initialize client
client = ElsClient(config['apikey'])
# client.inst_token = config['insttoken']

srch_line = 'fracture AND PMMA AND experiment OR test AND tension OR tensile'

In [2]:
# See https://github.com/ElsevierDev/elsapy/blob/master/elsapy/elssearch.py for the original execute function

from urllib.parse import quote_plus as url_encode

class myElsSearch(ElsSearch):
    def __init__(self, query, index, count = 25):
        super(myElsSearch, self).__init__(query, index)
        self._uri = self._base_url + self.index + '?count='+str(count)+'&query=' + url_encode(
                self.query)

    def execute(self, els_client=None, get_all=False):
        api_response = els_client.exec_request(self._uri)
        self._tot_num_res = int(api_response['search-results']['opensearch:totalResults'])
        self._results = api_response['search-results']['entry']
        if get_all is True:
            while (self.num_res < self.tot_num_res) and not self._upper_limit_reached():
                for e in api_response['search-results']['link']:
                    if e['@ref'] == 'next':
                        next_url = e['@href']
                api_response = els_client.exec_request(next_url)
                self._results += api_response['search-results']['entry']
                print(f'{self.num_res}/{self.tot_num_res}, {next_url}')
        self.results_df = recast_df(pd.DataFrame(self._results))

srch = myElsSearch('TITLE-ABS-KEY('+srch_line+')','scopus',count=200)
srch.execute(client, get_all=True)

print('Find', srch.num_res, 'results.')

df = pd.DataFrame(columns = ['doi', 'title', 'publication_name', 'url'])

for item in srch.results:
    try:
        item_df = pd.DataFrame({
            'doi': item['prism:doi'],
            'title': item['dc:title'],
            'publication_name': item['prism:publicationName'],
            'url': '=HYPERLINK(\"https://doi.org/'+item['prism:doi']+'\",\"DOI Link\")'
        }, index=[0])
        df = pd.concat([df,item_df], ignore_index=True)

    except:
        pass

df.reset_index()
df.to_excel('../data/'+srch_line+'.xlsx')
df.to_excel('../data/Extracted - '+srch_line+'.xlsx')

In [15]:
df

,doi,title,publication_name,url
0,10.1016/j.engfracmech.2022.108733,Investigation on brittle-ductile transition of...,Engineering Fracture Mechanics,NaN
1,10.1016/j.tafmec.2022.103282,Fracture testing and estimation of critical lo...,Theoretical and Applied Fracture Mechanics,NaN
2,10.1016/j.conbuildmat.2021.125998,The flame retardant and thermal conductivity p...,Construction and Building Materials,NaN
3,10.1007/s10856-022-06645-8,Characterization of various acrylate based art...,Journal of Materials Science: Materials in Med...,NaN
4,10.1016/j.tafmec.2021.103150,On the role of fracture process zone size in s...,Theoretical and Applied Fracture Mechanics,NaN
...,...,...,...,...
293,10.1007/BF00554938,Craze formation and growth in anisotropic poly...,Journal of Materials Science,NaN
294,10.1002/app.1975.070190603,Stress wave‐initiated fracture in amorphous th...,Journal of Applied Polymer Science,NaN
295,10.1007/BF00552926,Temperature effects in the fracture of PMMA,Journal of Materials Science,NaN
296,10.1007/BF00856367,Comparison of the long-term strengths of polym...,Polymer Mechanics,NaN
